In [1]:
import os
import gpxpy
import folium
import geopandas as gpd
from shapely.geometry import LineString
import tkinter as tk
from tkinter import messagebox
import webbrowser
import time
from geopy.distance import geodesic

# Path to your GPX files folder
gpx_folder = "/Users/chris/Desktop/Dektop/health/apple_health_export/workout-routes_updated"

# Threshold distance in meters to identify "unreasonable" jumps
DISTANCE_THRESHOLD_METERS = 200  # Adjust this based on your typical running pace

def load_gpx_track(file_path):
    """Load the track points from a GPX file and return filtered LineStrings to exclude large jumps."""
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
        segments = []

        for track in gpx.tracks:
            for segment in track.segments:
                points = []
                for i in range(1, len(segment.points)):
                    prev_point = segment.points[i - 1]
                    current_point = segment.points[i]
                    distance = geodesic((prev_point.latitude, prev_point.longitude), 
                                        (current_point.latitude, current_point.longitude)).meters
                    
                    # Check if the distance is within the threshold
                    if distance <= DISTANCE_THRESHOLD_METERS:
                        points.append((current_point.longitude, current_point.latitude))
                    else:
                        # Save the current segment and start a new one for large jumps
                        if len(points) > 1:
                            segments.append(LineString(points))
                        points = [(current_point.longitude, current_point.latitude)]  # Start new segment

                # Add the last segment if it has enough points
                if len(points) > 1:
                    segments.append(LineString(points))
    
    return segments  # Return all valid line segments

def create_map(gpx_folder):
    """Create a map with GPX tracks overlayed and display it in a browser."""
    m = folium.Map(location=[0, 0], zoom_start=2, tiles='cartodb positron')
    for filename in os.listdir(gpx_folder):
        if filename.endswith('.gpx'):
            file_path = os.path.join(gpx_folder, filename)
            track_segments = load_gpx_track(file_path)
            for segment in track_segments:
                gdf = gpd.GeoDataFrame(geometry=[segment], crs="EPSG:4326")
                for line in gdf.geometry:
                    folium.PolyLine(locations=[(lat, lon) for lon, lat in line.coords],
                                    color="blue", weight=2, opacity=0.6).add_to(m)
    
    # Save to a temporary file and open in the browser
    temp_path = "map_of_routes_temp.html"
    m.save(temp_path)

    # Wait a moment for the file to ensure Safari can access it
    time.sleep(0.5)  # Half a second delay
    webbrowser.get('safari').open(f"file://{os.path.abspath(temp_path)}")

def generate_map():
    """Generate the map and show a completion message."""
    create_map(gpx_folder)
    messagebox.showinfo("Map Generated", "Map displayed in your browser.")

# Setting up the GUI
app = tk.Tk()
app.title("GPX Map Overlay Application")
app.geometry("400x200")

# Adding button and label
label = tk.Label(app, text="Generate Map from GPX Tracks", font=("Helvetica", 14))
label.pack(pady=20)

button = tk.Button(app, text="Generate Map", command=generate_map)
button.pack(pady=10)

# Run the app
app.mainloop()

In [1]:
!pip install gpxpy folium geopandas shapely

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 5.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 5.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install geopy


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
